# Atividade 1 - Etapa 2 - Identificar os períodos de deglutição nos arquivos válidos

In [27]:
import pandas as pd

df = pd.read_csv('arquivos.csv')
df.set_index('0',inplace=True)
df.head()

,1,2,3,4,5,6,7,8,9,10,...,504,505,506,507,508,509,510,511,512,513
0,,,,,,,,,,,,,,,,,,,,,
a00001.mp3,0.619257,0.591173,0.588934,0.576050,0.569039,0.558396,0.591615,0.588218,0.560607,0.597746,...,0.001572,0.001675,0.001099,0.002113,0.002867,0.002273,0.002977,0.001725,0.000856,0.000328
a00002.mp3,0.579076,0.561442,0.574778,0.592735,0.597341,0.586346,0.579085,0.572200,0.559282,0.583274,...,0.005157,0.004259,0.004315,0.004707,0.003548,0.003606,0.003484,0.002382,0.001450,0.000089
a00003.mp3,0.659991,0.666785,0.614244,0.595386,0.595084,0.606930,0.569941,0.548508,0.509913,0.497616,...,0.000201,0.000238,0.000181,0.000245,0.000167,0.000218,0.000177,0.000133,0.000095,0.000008
a00004.mp3,0.595120,0.439680,0.370019,0.396822,0.355503,0.330981,0.379730,0.433511,0.469010,0.576902,...,0.000119,0.000113,0.000272,0.000157,0.000322,0.000315,0.000293,0.000348,0.000121,0.000015
a00005.mp3,0.600767,0.576361,0.621725,0.593066,0.625016,0.594525,0.563292,0.547832,0.565713,0.566939,...,0.004194,0.003753,0.002023,0.003272,0.002138,0.002041,0.001474,0.002276,0.000720,0.000108


## Classificador de Audios Válidos

Classificação geral dos arquivos de audio:

In [28]:
tutor = pd.read_csv('classificacao_anomalias.csv')
tutor = tutor[['File Name','Label']]
tutor['Label'] = tutor['Label'].apply(lambda x: 1 if x == 0 else 0)
tutor.set_index('File Name',inplace=True)
# identificando labels e quantidades
display(tutor['Label'].value_counts())
tutor.head()

Label
0    266
1    224
Name: count, dtype: int64

,Label
File Name,
a00001.mp3,1
a00002.mp3,0
a00003.mp3,1
a00004.mp3,1
a00005.mp3,1


In [29]:
# conectando a tabela de parâmetros da etapa 1 com os sons válidos do tutor
ds = df.join(tutor)
# removo as linhas que não tem identificação de válido
ds.dropna(inplace=True, axis=0)
display(ds.shape)
ds.head()

(490, 514)

,1,2,3,4,5,6,7,8,9,10,...,505,506,507,508,509,510,511,512,513,Label
0,,,,,,,,,,,,,,,,,,,,,
a00001.mp3,0.619257,0.591173,0.588934,0.576050,0.569039,0.558396,0.591615,0.588218,0.560607,0.597746,...,0.001675,0.001099,0.002113,0.002867,0.002273,0.002977,0.001725,0.000856,0.000328,1
a00002.mp3,0.579076,0.561442,0.574778,0.592735,0.597341,0.586346,0.579085,0.572200,0.559282,0.583274,...,0.004259,0.004315,0.004707,0.003548,0.003606,0.003484,0.002382,0.001450,0.000089,0
a00003.mp3,0.659991,0.666785,0.614244,0.595386,0.595084,0.606930,0.569941,0.548508,0.509913,0.497616,...,0.000238,0.000181,0.000245,0.000167,0.000218,0.000177,0.000133,0.000095,0.000008,1
a00004.mp3,0.595120,0.439680,0.370019,0.396822,0.355503,0.330981,0.379730,0.433511,0.469010,0.576902,...,0.000113,0.000272,0.000157,0.000322,0.000315,0.000293,0.000348,0.000121,0.000015,1
a00005.mp3,0.600767,0.576361,0.621725,0.593066,0.625016,0.594525,0.563292,0.547832,0.565713,0.566939,...,0.003753,0.002023,0.003272,0.002138,0.002041,0.001474,0.002276,0.000720,0.000108,1


Utilizando metodologias de aprendizado supervisionado classificador com validação cruzada (CV) para avaliação das técnicas com metrica F1:

In [30]:
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier, HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

X = ds.drop(columns=['Label']).values
y = ds['Label'].values

classificadores = [
    DecisionTreeClassifier(random_state=42,max_depth=5),
    ExtraTreeClassifier(random_state=42,max_depth=5),
    RandomForestClassifier(random_state=42,max_depth=5),
    ExtraTreesClassifier(random_state=42,max_depth=5),
    GradientBoostingClassifier(random_state=42,max_depth=5),
    AdaBoostClassifier(random_state=42),
    HistGradientBoostingClassifier(random_state=42,max_depth=5),
    LogisticRegression(random_state=42),
]

resultados = []
for cls in tqdm(classificadores):
    res = cross_validate(cls, X, y, cv=5, scoring='f1')
    resultados.append(
        {'metodo': cls.__class__.__name__, 
         'f1': res['test_score'].mean(), 
         'tempo': res['fit_time'].mean(),
         }
    )

warnings.filterwarnings('default')
df_res = pd.DataFrame(resultados)
df_res.sort_values('f1', ascending=False)

  0%|          | 0/8 [00:00<?, ?it/s]

100%|██████████| 8/8 [01:12<00:00,  9.02s/it]


,metodo,f1,tempo
2,RandomForestClassifier,0.614889,0.445884
3,ExtraTreesClassifier,0.612670,0.114525
7,LogisticRegression,0.604044,0.057804
6,HistGradientBoostingClassifier,0.583225,1.150490
1,ExtraTreeClassifier,0.579061,0.001404
4,GradientBoostingClassifier,0.554420,11.143267
5,AdaBoostClassifier,0.548514,1.357565
0,DecisionTreeClassifier,0.540769,0.113886


In [32]:
clf = RandomForestClassifier(random_state=42,max_depth=5)
clf.fit(X, y)
clf.score(X,y)

0.863265306122449

In [37]:
from IPython.display import Image, Audio, display
from ipywidgets import widgets

ds.reset_index(inplace=True)

blocos = []
for i, row in ds.head(50).iterrows():
    nome = row['0'].split('.')[0]
    out = widgets.Output()
    with out:
        display(Image(f'dataset/Charts/{nome}.png'))
        display('Inválido' if row['Label'] == 0 else 'Válido')
        display(Audio(f'dataset/MP3/{nome}.mp3'))
    blocos.append(out)
widgets.HBox(blocos)